In [30]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, accuracy_score, mean_squared_error
from matplotlib.pyplot import figure
from keras.layers import Conv1D, MaxPooling1D, ConvLSTM1D, Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.layers import Activation, Flatten, Dense, Dropout, LSTM, Concatenate, Input, Bidirectional, RepeatVector, Reshape, GaussianNoise
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras import regularizers
import tensorflow as tf
from keras.utils import np_utils
from tensorflow.keras.models import Model
from torch.utils.data import DataLoader
from torch.optim import Adam
import torch.nn as nn
import torch
import random
import math
import scipy as sp
import numpy as np
import scipy.io
from google.colab import drive 
from scipy.fftpack import fft
import cmath
from math import log
from scipy import fftpack
from numpy import sum,isrealobj,sqrt
from numpy.random import standard_normal
from numpy import savetxt
from google.colab import drive 
drive.mount('/content/gdrive')
################################################################################
def radioFadingChannel(nSample): 
  v_mph = 60 # velocity of either TX or RX, in miles per hour
  center_freq = 200e6 # RF carrier frequency in Hz
  Fs = nSample # sample rate of simulation
  N = 100 # number of sinusoids to sum
  v = v_mph * 0.44704 # convert to m/s
  fd = v*center_freq/3e8 # max Doppler shift
  t = np.arange(0, 1, 1/Fs) # time vector. (start, stop, step)*
  x = np.zeros(len(t))
  y = np.zeros(len(t))
  for i in range(N):
    alpha = (np.random.rand() - 0.5) * 2 * np.pi
    phi = (np.random.rand() - 0.5) * 2 * np.pi
    x = x + np.random.randn() * np.cos(2 * np.pi * fd * t * np.cos(alpha) + phi)
    y = y + np.random.randn() * np.sin(2 * np.pi * fd * t * np.cos(alpha) + phi)
  # z is the complex coefficient representing channel
  z = (1/np.sqrt(N)) * (x + 1j*y) 
  z_mag = np.abs(z)
  return z
####################### Functions ############################################## 
def quadMean(x):
    y = np.array([abs(n)**2 for n in x])
    return y
###
def setMeanPower(signal, Power):
  y=0
  pZero = quadMean(signal)
  alpha = np.sqrt(Power/pZero)
  y = alpha*signal
  return y
###
def setSNR(signal, h, SNRdB):
  r = len(signal)
  y = np.zeros(r)
  snrSlin = 10**(SNRdB/10)
  pSignal = quadMean(h*signal)
  pNoise = pSignal/snrSlin     
  n = standard_normal(len(signal))+1j*standard_normal(len(signal))
  noise = standard_normal(len(signal))+1j*standard_normal(len(signal))
  n = setMeanPower(n, pNoise)
  noise = setMeanPower(noise, pNoise)
  y = h*signal + n
  return y, noise
################################################################################
L = 58050000
N = 100
M = 15
S = 3
SNR = np.arange(-20,1, 1)
signal = standard_normal(L)+1j*standard_normal(L)
H = radioFadingChannel(len(signal[1])) 
Hot = np.array([setSNR(signal, H, SNR[i]) for i in range(len(SNR))])
H1t, H0t = Hot[:, 0,:], Hot[:, 1,:]
H1t, H0t = H1t.reshape(len(SNR), int(L/N), N), H0t.reshape(len(SNR), int(L/N), N)
np.random.seed(32)
for i in range(H1t.shape[0]):
  np.random.shuffle(H1t[i])
  np.random.shuffle(H0t[i])
H1, H0 = H1t.reshape(len(SNR),int(L/(M*N*S)), N*M*S), H0t.reshape(len(SNR),int(L/(M*N*S)), N*M*S)
H1, H0 = np.abs(H1), np.abs(H0)
i = np.arange(S+1)*N*M
H1x, H0x = H1[:,:,i[0]:i[1]], H0[:,:,i[0]:i[1]]
H1y, H0y = H1[:,:,i[1]:i[2]], H0[:,:,i[1]:i[2]]
H1z, H0z = H1[:,:,i[2]:i[3]], H0[:,:,i[2]:i[3]]
################################################################################
def matrix_generation(X1, X0, length, n_exemples): 
  p = int(n_exemples/2)
  X1=X1[0:p,:]
  X0=X0[0:p,:]
  X1 = X1.reshape(X1.shape[0], 15, 100)
  X0 = X0.reshape(X0.shape[0], 15, 100)
  H1 = (1/length)*np.array([X1[i] @ X1[i].T for i in range(X1.shape[0])])
  H0 = (1/length)*np.array([X0[i] @ X0[i].T for i in range(X0.shape[0])])
  y1 = np.ones((p,1))
  y0 = np.zeros((p,1))
  X, y = np.vstack((X1,X0)), np.vstack((y1,y0))
  X = X.reshape(X.shape[0], length, length, 1).astype('float32')
  # Data splitting  
  Xc_train, _, yc_train, _ = train_test_split(X, y, random_state=42, train_size=1.0) 
  # Encoding Categorical Data
  Yc_train = np_utils.to_categorical(yc_train, 2)
  return Xc_train, Yc_train
################################################################################
length = 15
n_exemples = 24000
for i in range(len(SNR)):
  x, Yc_test = matrix_generation(H1x[i,:], H0x[i,:], length, n_exemples)
  y,_ = matrix_generation(H1y[i,:], H0y[i,:], length, n_exemples)
  z,_ = matrix_generation(H1z[i,:], H0z[i,:], length, n_exemples)
  globals()['Xc_test%s' % SNR[i]] =  np.vstack((x, y, z))
################################################################################
np.save('/content/gdrive/My Drive/Xc20_testP.npy', Xc_test20)
np.save('/content/gdrive/My Drive/Xc19_testP.npy', Xc_test19)
np.save('/content/gdrive/My Drive/Xc18_testP.npy', Xc_test18)
np.save('/content/gdrive/My Drive/Xc17_testP.npy', Xc_test17)
np.save('/content/gdrive/My Drive/Xc16_testP.npy', Xc_test16)
np.save('/content/gdrive/My Drive/Xc15_testP.npy', Xc_test15)
np.save('/content/gdrive/My Drive/Xc14_testP.npy', Xc_test14)
np.save('/content/gdrive/My Drive/Xc13_testP.npy', Xc_test13)
np.save('/content/gdrive/My Drive/Xc12_testP.npy', Xc_test12)
np.save('/content/gdrive/My Drive/Xc11_testP.npy', Xc_test11)
np.save('/content/gdrive/My Drive/Xc10_testP.npy', Xc_test10)
np.save('/content/gdrive/My Drive/Xc9_testP.npy', Xc_test9)
np.save('/content/gdrive/My Drive/Xc8_testP.npy', Xc_test8)
np.save('/content/gdrive/My Drive/Xc7_testP.npy', Xc_test7)
np.save('/content/gdrive/My Drive/Xc6_testP.npy', Xc_test6)
np.save('/content/gdrive/My Drive/Xc5_testP.npy', Xc_test5)
np.save('/content/gdrive/My Drive/Xc4_testP.npy', Xc_test4)
np.save('/content/gdrive/My Drive/Xc3_testP.npy', Xc_test3)
np.save('/content/gdrive/My Drive/Xc2_testP.npy', Xc_test2)
np.save('/content/gdrive/My Drive/Xc1_testP.npy', Xc_test1)
np.save('/content/gdrive/My Drive/Xc0_testP.npy', Xc_test0)
np.save('/content/gdrive/My Drive/Yc_testP.npy', Yc_test)

1